In [288]:
# import os
# import math
# import tflearn
# import pandas as pd
# import nltk
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
# from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
# import sklearn.linear_model as model
import csv
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import GridSearchCV
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import SGDClassifier
# import sklearn
# from sklearn.naive_bayes import BernoulliNB
from tflearn.data_utils import to_categorical, pad_sequences
# from tflearn.datasets import imdb
import tensorflow as tf 
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout

### Basic Neural Network

Use Michelle's method for importing data since I can't do it correctly:

In [211]:
train_data = np.genfromtxt('Data/training_data.txt',dtype='str')
test_data = np.genfromtxt('Data/test_data.txt',dtype='str')
#train_data = np.loadtxt('training_data.txt',  delimiter=' ')
#test_data = np.loadtxt('test_data.txt')
train_labels = train_data[0, :]
train_stars = train_data[1:, 0]
train_reviews = train_data[1:, 1:]

test_labels = test_data[0, :]
#test_stars = train_data[1:, 0]
test_reviews = test_data[1:, 0:]

trainX = train_reviews
trainY = train_stars
testX = test_reviews
#testY = test_stars

print("trainX = ", trainX.shape)
print("trainY = ", trainY.shape)
print("testX = ", testX.shape)


for i in range(0, len(trainX)):
    for j in range(0, len(trainX[0])):
        trainX[i][j] = int(trainX[i][j])

for i in range(0, len(testX)):
    for j in range(0, len(testX[0])):
        testX[i][j] = int(testX[i][j])

for i in range(0, len(trainY)):
        trainY[i] = int(trainY[i])
#for i in range(0, len(trainY)):
        #testY[i] = int(testY[i])

trainX =  (20000, 1000)
trainY =  (20000,)
testX =  (10000, 1000)


Define a function for classification error:

In [227]:
def class_error(y, y_pred):
    misclassified = 0
    for i in range(len(y)):
        if y[i] != y_pred[i]:
            misclassified += 1
    return float(misclassified) / len(y)

Split data into 80% training set and 20% test set.

In [247]:
num_rows = []
for i in range(len(trainY)):
    num_rows.append(i)
    
np.random.shuffle(num_rows)
train_indices = num_rows[:16000]
valid_indices = num_rows[16000:]

train_X = []
train_y = []
valid_X = []
valid_y = []

for index in train_indices:
    train_X.append(trainX[index])
    train_y.append(trainY[index])
    
for index in valid_indices:
    valid_X.append(trainX[index])
    valid_y.append(trainY[index])

With Michelle's code, it doesn't actually convert the array contents into integers because she tried to do it in place. This code does it correctly out of place:

In [248]:
train_X2 = []
for row in train_X:
    temp = []
    for element in row:
        temp.append(int(element))
    train_X2.append(temp)

valid_X2 = []
for row in valid_X:
    temp = []
    for element in row:
        temp.append(int(element))
    valid_X2.append(temp)
    
valid_y2 = []
for row in valid_y:
    valid_y2.append(int(row))
    
train_y2 = []
for row in train_y:
    train_y2.append(int(row))
    
test_X2 = []
for row in testX:
    temp = []
    for element in row:
        temp.append(int(element))
    test_X2.append(temp)

Convert everything back to a numpy array:

In [249]:
train_X2 = np.array(train_X2)
valid_X2 = np.array(valid_X2)
train_y2 = np.array(train_y2)
valid_y2 = np.array(valid_y2)
test_X2 = np.array(test_X2)

# Basic Neural Network 

In [235]:
print(valid_X2)

[[1 0 0 ... 0 0 0]
 [4 0 5 ... 0 0 0]
 [2 0 0 ... 0 0 0]
 ...
 [1 0 1 ... 0 0 0]
 [2 1 3 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [236]:
print(validY)

[[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]


In [251]:
train_y2 = to_categorical(train_y2, 2)

In [141]:
print(train_y2)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [0. 1.]]


In [252]:
valid_y2 = to_categorical(valid_y2, 2)
print(valid_y2)

[[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]


In [334]:
model = Sequential()

keras.layers.BatchNormalization(axis=-1, 
momentum=0.99, epsilon=0.001, center=True, 
scale=True, beta_initializer='zeros', 
gamma_initializer='ones', moving_mean_initializer='zeros', 
moving_variance_initializer='ones', beta_regularizer=None, 
gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)

# change input dim to 1000 for 
model.add(Dense(200, input_dim=1000))
# keras.layers.LeakyReLU(alpha=0.3)
model.add(Activation('sigmoid'))
model.add(Dropout(0.10))

model.add(Dense(200)) 
model.add(Activation('sigmoid'))
# keras.layers.LeakyReLU(alpha=0.3)
model.add(Dropout(0.10))

model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

# with tf idf
# train_XTF = TFIDF(train_X2, train_labels)
# valid_XTF = TFIDF(valid_X2, train_labels)
# print(np.shape(valid_XTF[:, :1000]), np.shape(valid_X2))

# fit = model.fit(train_XTF[:, :1000], train_y2, batch_size=128, nb_epoch=10, validation_data=(valid_XTF[:, :1000], valid_y2),
#     verbose=1)


# without tfidf
fit = model.fit(train_X2, train_y2, batch_size=128, nb_epoch=12, validation_data=(valid_X2, valid_y2),
  verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_267 (Dense)            (None, 200)               200200    
_________________________________________________________________
activation_258 (Activation)  (None, 200)               0         
_________________________________________________________________
dropout_177 (Dropout)        (None, 200)               0         
_________________________________________________________________
dense_268 (Dense)            (None, 200)               40200     
_________________________________________________________________
activation_259 (Activation)  (None, 200)               0         
_________________________________________________________________
dropout_178 (Dropout)        (None, 200)               0         
_________________________________________________________________
dense_269 (Dense)            (None, 2)                 402       
__________

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 16000 samples, validate on 4000 samples
Epoch 1/12
16000/16000 [==============================] - 4s 267us/step - loss: 0.5994 - acc: 0.6803 - val_loss: 0.3983 - val_acc: 0.8488
Epoch 2/12
16000/16000 [==============================] - 1s 88us/step - loss: 0.3719 - acc: 0.8373 - val_loss: 0.3428 - val_acc: 0.8592
Epoch 3/12
16000/16000 [==============================] - 1s 90us/step - loss: 0.3366 - acc: 0.8576 - val_loss: 0.3391 - val_acc: 0.8568
Epoch 4/12
16000/16000 [==============================] - 1s 92us/step - loss: 0.3291 - acc: 0.8592 - val_loss: 0.3436 - val_acc: 0.8585
Epoch 5/12
16000/16000 [==============================] - 1s 91us/step - loss: 0.3216 - acc: 0.8607 - val_loss: 0.3446 - val_acc: 0.8538
Epoch 6/12
16000/16000 [==============================] - 1s 90us/step - loss: 0.3177 - acc: 0.8649 - val_loss: 0.3431 - val_acc: 0.8575
Epoch 7/12
16000/16000 [==============================] - 2s 96us/step - loss: 0.3161 - acc: 0.8661 - val_loss: 0.3425 - val_acc

Fit the model:

In [181]:
print(np.shape(train_XTF[:, :1000]))

(16000, 1000)


Print the classification error on the training set:

In [335]:
# prediction = model.predict(train_XTF[:, :1000])
prediction = model.predict(train_X2)
predictedY = []

trainY_new = []
for i in range(0, len(train_y2)):
    if train_y2[i][0] == 1:
        trainY_new.append(0)
    else:
        trainY_new.append(1)

for i in range(0, len(prediction)):
    if abs(prediction[i][0]-1) < abs(prediction[i][1]-1):
        predictedY.append(0)
    else:
        predictedY.append(1)
        
class_error(predictedY, trainY_new)

0.122

In [336]:
# prediction_V = model.predict(valid_XTF[:, :1000])
# print(valid_X2)

prediction_V = model.predict(valid_X2)
predictedY_V = []

validY_new = []
for i in range(0, len(valid_y2)):
    if valid_y2[i][0] == 1:
        validY_new.append(0)
    else:
        validY_new.append(1)

for i in range(0, len(prediction_V)):
    if abs(prediction_V[i][0] - 1) < abs(prediction_V[i][1] - 1):
        predictedY_V.append(0)
    else:
        predictedY_V.append(1)

class_error(predictedY_V, validY_new)

0.14475

In [310]:
testXTF = TFIDF(test_X2, test_labels)
# print(np.shape(test_X2), np.shape(test_labels), np.shape(train_labels), np.shape(train_X2))
# print(np.shape(testXTF))

In [337]:
test_predictions = model.predict(test_X2)
# test_predictions = model.predict(testXTF)
predictions_arr = []

for i in range(0, len(test_predictions)):
    if abs(test_predictions[i][0]-1) < abs(test_predictions[i][1]-1):
        predictions_arr.append(0)
    else:
        predictions_arr.append(1)
    
writeToText(predictions_arr, 'basicNeuralNetNOTF.txt')

Print the classification error on the validation set:

A function for writing to the text file for the submission:

In [262]:
def writeToText(predictions, filename):
    array = [["Id","Prediction"]]
    index = 1
    for i in range(len(predictions)):
        predict = predictions[index - 1]
        array.append([index, int(predictions[index - 1])])
        index += 1
    f = open(filename, 'w')
    writer = csv.writer(f, delimiter=',', quotechar='|')
    writer.writerows(array)

### Logistic Regression with TF-IDF

A function to convert the set into a TF-IDF matrix:

In [300]:
def TFIDF(X, labels):
    documents = []

    for element in X:
        temp = ''
        for i in range(len(element) - 1):
            word = labels[i + 1] + ' '
            new_word = word * element[i]
            temp += (new_word)
        documents.append(temp)
        
    tokenize = lambda doc: doc.lower().split(" ")
    sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)
    return sklearn_tfidf.fit_transform(documents)

Fit the model using the training data:

In [ ]:
model = Sequential()
model.add(Dense(1000, input_dim=1000))
model.add(Activation('relu'))
model.add(Dropout(0.20))

model.add(Dense(1000)) 
model.add(Activation('relu'))
model.add(Dropout(0.20))

model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

inputX = TFIDF(train_X2, train_labels)
print(inputX)
print(len(inputX))

fit = model.fit(inputX, trainY, batch_size=64, nb_epoch=15, validation_data=(validX, validY),
    verbose=1)


prediction = model.predict(trainX)
predictedY = []

trainY_new = []
for i in range(0, len(trainY)):
    if trainY[i][0] == 1:
        trainY_new.append(0)
    else:
        trainY_new.append(1)



for i in range(0, len(prediction)):
    if abs(prediction[i][0]-1) < abs(prediction[i][1]-1):
        predictedY.append(0)
    else:
        predictedY.append(1)
        
class_error(predictedY, trainY_new)